# Formula 1 Qualifying vs Fastest Lap Time comparison
In 2019 the FIA (governing body of F1) changed the rules to award an extra point to the driver in the top 10 finishing positions (the only positions that are awarded points) who sets the fastest lap in the race.  There were a lot of questions as to how this would affect teams strategy.  I was curious to know how frequently a driver in the top 10 would be capable of setting the fastest lap.  

In F1 cars start the race with a maximum of 105kg of fuel and there is no refueling, during the race the cars stop for one or more pit stops for fresh tires.  Cars set faster and faster laps as the race goes on for two reasons, they are burning off fuel and the track is becoming stickier because rubber deposition on the racing line.  A car could set it's fastest theoretical lap by pitting with about 4 laps left for fresh tires and pushing hard on a light fuel load.  Pitting late in the race could allow a competitor to pass if the car hadn't built up enough of a lead.  

In qualifying cars run laps with fresh tires and low fuel.  I wanted to see how frequently backmarker teams had set a faster qualifying time than the fastest lap of the race.  There are three qualifying sessions Q1, Q2, and Q3 with every car, top 15, and top 10 fastest cars participating respectively.  In this notebook I compare the number of Q1 times that were slower than the fastest lap of the race (times where the slowest team hadn't set a faster lap in qualifying than the fastest lap in the race)

Data pulled from [Ergast](http://ergast.com)

In [1]:
#only run this cell if you ndeed to download the data, uncomment appropriately
!#wget http://ergast.com/downloads/f1db_csv.zip
!#unzip f1db_csv.zip

In [2]:
import pandas as pd
import numpy as np

In [3]:
#data frame creating transform functions
#playing with globals is a bit hacky, but it works for this exmaple.  this could easily be modified to write
#to a data registry
def read_csv(fname, output_df_name, **csv_args):
    out_df = pd.read_csv(fname, **csv_args)
    globals()[output_df_name] = out_df
def join(left_df, right_df, output_df_name, **join_args):
    out_df = left_df.join(right_df, **join_args)
    globals()[output_df_name] = out_df

In [4]:
#transforms defined here, first argument is always df
#second argument from_col(s), third argument to_col(s)
#fourth argument kw_args
def col_drop(df, cols):
    for col in cols:
        df.drop(col, axis=1, inplace=True)
def create_ts_col(df, from_col, to_col, **date_time_args):
    from_ser = df[from_col]
    df[to_col] = pd.to_datetime(from_ser, **date_time_args)
def subtract(df, minuend_col, to_col, subtrahend):
    df[to_col] = df[minuend_col] - subtrahend
def set_index(df, col):
    df.set_index(col, inplace=True)
def inplace_update(df, col, func):
    df[col] = df[col].map(func)

In [5]:
read_csv("lap_times.csv", "ldf", names=["raceId", "driverId", "lap" , "position", "time", "milliseconds"])
create_ts_col(ldf, 'milliseconds', 'lap_time_epoch', unit='ms')
subtract(ldf,  'lap_time_epoch', 'lap_time', pd.to_datetime(0, unit="ms"))
col_drop(ldf, ['lap_time_epoch', 'time', 'milliseconds'])
ldf.head(2)

,raceId,driverId,lap,position,lap_time
0,841,20,1,1,00:01:38.109000
1,841,20,2,1,00:01:33.006000


In [6]:
read_csv("driver.csv", "ddf",
    names=["driverId", "driverRef", "number", "code",
           "forename", "surname", "dob", "nationality",  "url"])
col_drop(ddf, ['number', 'forename', 'surname', 'nationality', 'dob', 'url'])
set_index(ddf, 'driverId')

In [7]:
rdf = race_df = pd.read_csv("races.csv", 
                      names=["raceId", "year", "round", "circuitId", "name", "date", "time", "url"])
col_drop(rdf, ['circuitId', 'date', 'time', 'url'])
set_index(rdf, 'raceId')

In [8]:
join(ldf, ddf, 'comb_df', on="driverId", how="inner", lsuffix="_lap", rsuffix="_drv")
join(comb_df, rdf, 'comb_df2', on="raceId", how="inner", lsuffix="_lap", rsuffix="_rac")
comb_df2.head()  # this head call is only necessary to check what's going on, in a gui this would be an expand
                 # button on a node

,raceId,driverId,lap,position,lap_time,driverRef,code,year,round,name
0,841,20,1,1,00:01:38.109000,vettel,VET,2011,1,Australian Grand Prix
1,841,20,2,1,00:01:33.006000,vettel,VET,2011,1,Australian Grand Prix
2,841,20,3,1,00:01:32.713000,vettel,VET,2011,1,Australian Grand Prix
3,841,20,4,1,00:01:32.803000,vettel,VET,2011,1,Australian Grand Prix
4,841,20,5,1,00:01:32.342000,vettel,VET,2011,1,Australian Grand Prix


In [9]:
#Off the top of my head I don't know how to write the next line as a set of composable transforms
fastest_laps = comb_df2.loc[comb_df2.groupby(['year', 'round'])["lap_time"].idxmin()]
fastest_laps.tail()

,raceId,driverId,lap,position,lap_time,driverRef,code,year,round,name
443685,1005,20,53,6,00:01:32.318000,vettel,VET,2018,17,Japanese Grand Prix
444545,1006,1,40,3,00:01:37.392000,hamilton,HAM,2018,18,United States Grand Prix
445719,1007,822,65,5,00:01:18.741000,bottas,BOT,2018,19,Mexican Grand Prix
446758,1008,822,65,5,00:01:10.540000,bottas,BOT,2018,20,Brazilian Grand Prix
448104,1009,20,54,2,00:01:40.867000,vettel,VET,2018,21,Abu Dhabi Grand Prix


In [10]:
read_csv("qualifying.csv", "qdf", 
         names=["qualifyId", "raceId", "qdriverId", "constructorId", "number",
                "qposition", "q1", "q2", "q3"])
col_drop(qdf, ['qualifyId', 'constructorId', 'number'])

In [11]:
set_index(fastest_laps, 'raceId')
join(qdf, fastest_laps, 'rqd_1', on="raceId", how="inner")
join(rqd_1, ddf, 'rqd', on="qdriverId", how="inner", lsuffix="_fas", rsuffix="_qual")
col_drop(rqd, ['raceId', 'qdriverId', 'position', 'driverId', 'driverRef_qual', 'driverRef_fas'])

In [12]:
def normalize_q(q_str):
    if q_str == "\\N":
        return np.nan
    if str(q_str) == "nan":
        return np.nan
    return "00:0%s" % q_str

In [13]:
inplace_update(rqd, 'q1', normalize_q)
inplace_update(rqd, 'q2', normalize_q)
inplace_update(rqd, 'q3', normalize_q)

In [14]:
create_ts_col(rqd, 'q1', 'q1_tdm')
create_ts_col(rqd, 'q2', 'q2_tdm')
create_ts_col(rqd, 'q3', 'q3_tdm')
subtract(rqd, 'q1_tdm', 'q1_td', pd.to_datetime('00:00:00'))
subtract(rqd, 'q2_tdm', 'q2_td', pd.to_datetime('00:00:00'))
subtract(rqd, 'q3_tdm', 'q3_td', pd.to_datetime('00:00:00'))
col_drop(rqd, ['q1', 'q2', 'q3', 'q1_tdm', 'q2_tdm', 'q3_tdm'])

In [15]:
#once again I'm not really sure how to cleanly express the following cells as transforms

In [16]:
faster_q1 = rqd[rqd['q1_td'] < rqd['lap_time']]

In [17]:
faster_q1.groupby(['year', 'round']).max()['qposition'].tail(25)

year  round
2017  17       20
      18       15
      19       19
      20       20
2018  1        20
      2        20
      3        20
      4        17
      5        10
      6        19
      7        18
      8        20
      9        20
      10       18
      11       20
      12       20
      13       20
      14       20
      15       20
      16       18
      17       20
      18       20
      19       20
      20       20
      21       20
Name: qposition, dtype: int64

In [18]:
slower_q1 = rqd[rqd['q1_td'] > rqd['lap_time']]
slower_q1_2016 = slower_q1[slower_q1['year'] > 2016]
slower_q1_2016.set_index(['year', 'round', 'name'], inplace=True)
slower_q1_2016

qposition  lap        lap_time code_fas  \
year round name                                                             
2017 10    British Grand Prix             1   48 00:01:30.621000      HAM   
     13    Italian Grand Prix             1   49 00:01:23.361000      RIC   
     10    British Grand Prix            15   48 00:01:30.621000      HAM   
     13    Italian Grand Prix             9   49 00:01:23.361000      RIC   
     10    British Grand Prix             3   48 00:01:30.621000      HAM   
     13    Italian Grand Prix             8   49 00:01:23.361000      RIC   
     8     Azerbaijan Grand Prix         16   47 00:01:43.441000      VET   
     10    British Grand Prix            13   48 00:01:30.621000      HAM   
     13    Italian Grand Prix            13   49 00:01:23.361000      RIC   
     10    British Grand Prix             2   48 00:01:30.621000      HAM   
     13    Italian Grand Prix             7   49 00:01:23.361000      RIC   
     4     Russian Grand Prix            20   49 00:01:36.844000      RAI   
     8     Azerbaijan Grand Prix         17   47 00:01:43.441000      VET   
     10    British Grand Prix            10   48 00:01:30.621000      HAM   
     13    Italian Grand Prix            20   49 00:01:23.361000      RIC   
     18    Mexican Grand Prix            19   68 00:01:18.785000      VET   
     8     Azerbaijan Grand Prix         14   47 00:01:43.441000      VET   
     10    British Grand Prix             6   48 00:01:30.621000      HAM   
     13    Italian Grand Prix            12   49 00:01:23.361000      RIC   
2018 5     Spanish Grand Prix            16   61 00:01:18.441000      RIC   
2017 10    British Grand Prix             7   48 00:01:30.621000      HAM   
     13    Italian Grand Prix            11   49 00:01:23.361000      RIC   
2018 5     Spanish Grand Prix            15   61 00:01:18.441000      RIC   
2017 10    British Grand Prix            20   48 00:01:30.621000      HAM   
     13    Italian Grand Prix             3   49 00:01:23.361000      RIC   
     10    British Grand Prix             4   48 00:01:30.621000      HAM   
     13    Italian Grand Prix             6   49 00:01:23.361000      RIC   
     1     Australian Grand Prix         17   56 00:01:26.538000      RAI   
     3     Bahrain Grand Prix            20   46 00:01:32.798000      HAM   
     8     Azerbaijan Grand Prix         13   47 00:01:43.441000      VET   
...                                     ...  ...             ...      ...   
           Azerbaijan Grand Prix         15   47 00:01:43.441000      VET   
     10    British Grand Prix            18   48 00:01:30.621000      HAM   
     12    Belgian Grand Prix            20   41 00:01:46.577000      VET   
     13    Italian Grand Prix            19   49 00:01:23.361000      RIC   
     14    Singapore Grand Prix          19   55 00:01:45.008000      HAM   
     18    Mexican Grand Prix            17   68 00:01:18.785000      VET   
     1     Australian Grand Prix         18   56 00:01:26.538000      RAI   
     4     Russian Grand Prix            17   49 00:01:36.844000      RAI   
     8     Azerbaijan Grand Prix         19   47 00:01:43.441000      VET   
     10    British Grand Prix             9   48 00:01:30.621000      HAM   
     13    Italian Grand Prix            10   49 00:01:23.361000      RIC   
2018 5     Spanish Grand Prix            11   61 00:01:18.441000      RIC   
     16    Russian Grand Prix            19   50 00:01:35.861000      BOT   
2017 2     Chinese Grand Prix            20   44 00:01:35.378000      HAM   
     10    British Grand Prix             8   48 00:01:30.621000      HAM   
     13    Italian Grand Prix             5   49 00:01:23.361000      RIC   
2018 5     Spanish Grand Prix            13   61 00:01:18.441000      RIC   
2017 1     Australian Grand Prix         19   56 00:01:26.538000      RAI   
     6     Monaco Grand Prix             18   76 00:01:14.820000      PER   
     8     Azerbaijan Grand Pr

In [19]:
len(rqd[rqd['year'] > 2016])

818

# Conclusion
## Almost every driver would be capable of setting the fastest lap in a race with a late pitstop

Out of 818 q1 qualifying laps, only 89 were slower than fastest lap of the race.  After investigating a couple of the races where this occured, I found that qualifying took place in the rain.